# Traveling Ethiopia Search Problem Question #2

To solve this problem, we represent the state space graph (Figure 2) using an Adjacency Dictionary where every city maps to a list of tuples: (neighbor_city, cost).Static Data: The map itself is stored in a Dictionary.Dynamic Data (Search): For Uniform Cost Search (UCS), we use a Priority Queue. This is the most "manageable data structure" for this specific algorithm because it automatically sorts paths by lowest cumulative cost (path cost $g(n)$).

The following code block is a complete implementation. It includes the weighted graph representation, the standard Uniform Cost Search, and the Customized Multi-Goal Search strategy.

In [ ]:
import heapq

# 1. DATA REPRESENTATION (Figure 2 converted to Adjacency Dictionary with Costs)
# Format: { 'City': {'Neighbor': Cost, ...}, ... }
# Note: I have transcribed the costs from Figure 2. For the few edges where
# numbers were obscured, I used logical estimates based on visual length.

ethiopia_map_weighted = {
    'Addis Ababa': {'Adama': 3, 'Ambo': 5, 'Debre Birhan': 5},
    'Adama': {'Addis Ababa': 3, 'Matahara': 3, 'Asella': 4, 'Batu': 4},
    'Ambo': {'Addis Ababa': 5, 'Wolkite': 6, 'Nekemte': 9},
    'Debre Birhan': {'Addis Ababa': 5, 'Debre Sina': 2},
    'Debre Sina': {'Debre Birhan': 2, 'Kemise': 7, 'Debre Markos': 17},
    'Debre Markos': {'Debre Sina': 17, 'Finote Selam': 3},
    'Kemise': {'Debre Sina': 7, 'Dessie': 4},
    'Dessie': {'Kemise': 4, 'Woldia': 6},
    'Woldia': {'Dessie': 6, 'Lalibela': 7, 'Alamata': 3, 'Samara': 8},
    'Lalibela': {'Woldia': 7, 'Debre Tabor': 8, 'Sekota': 6},
    'Sekota': {'Lalibela': 6, 'Mekelle': 9, 'Alamata': 6},
    'Debre Tabor': {'Lalibela': 8, 'Bahir Dar': 4},
    'Bahir Dar': {'Debre Tabor': 4, 'Finote Selam': 6, 'Injibara': 4, 'Metekel': 11, 'Azezo': 7},
    'Finote Selam': {'Bahir Dar': 6, 'Debre Markos': 3, 'Injibara': 2},
    'Injibara': {'Bahir Dar': 4, 'Finote Selam': 2},
    'Metekel': {'Bahir Dar': 11},
    'Azezo': {'Bahir Dar': 7, 'Gondar': 1, 'Metema': 7},
    'Gondar': {'Azezo': 1, 'Humera': 9, 'Metema': 7, 'Debarke': 4},
    'Metema': {'Azezo': 7, 'Gondar': 7, 'Kartum': 19},
    'Kartum': {'Metema': 19, 'Humera': 21},
    'Humera': {'Kartum': 21, 'Gondar': 9, 'Shire': 8},
    'Shire': {'Humera': 8, 'Debarke': 2, 'Axum': 2},
    'Debarke': {'Gondar': 4, 'Shire': 2},
    'Axum': {'Shire': 2, 'Adwa': 1, 'Asmara': 5},
    'Adwa': {'Axum': 1, 'Adigrat': 4, 'Mekelle': 7},
    'Asmara': {'Axum': 5, 'Adigrat': 6},
    'Adigrat': {'Asmara': 6, 'Adwa': 4, 'Mekelle': 7},
    'Mekelle': {'Adwa': 7, 'Adigrat': 7, 'Sekota': 9, 'Alamata': 5},
    'Alamata': {'Mekelle': 5, 'Sekota': 6, 'Woldia': 3, 'Samara': 11},
    'Samara': {'Woldia': 8, 'Alamata': 11, 'Fanti Rasu': 7, 'Gabi Rasu': 9},
    'Fanti Rasu': {'Samara': 7, 'Kilbet Rasu': 6},
    'Kilbet Rasu': {'Fanti Rasu': 6},
    'Gabi Rasu': {'Samara': 9, 'Awash': 5},
    'Awash': {'Gabi Rasu': 5, 'Matahara': 1, 'Chiro': 4},
    'Matahara': {'Adama': 3, 'Awash': 1},
    'Chiro': {'Awash': 4, 'Dire Dawa': 8},
    'Dire Dawa': {'Chiro': 8, 'Harar': 4},
    'Harar': {'Dire Dawa': 4, 'Babile': 2},
    'Babile': {'Harar': 2, 'Jigjiga': 3},
    'Jigjiga': {'Babile': 3, 'Dega Habur': 5},
    'Dega Habur': {'Jigjiga': 5, 'Kebri Dehar': 6},
    'Kebri Dehar': {'Dega Habur': 6, 'Gode': 5, 'Werder': 6},
    'Werder': {'Kebri Dehar': 6},
    'Gode': {'Kebri Dehar': 5, 'Dollo': 17, 'Mokadisho': 22, 'Sof Oumer': 23},
    'Dollo': {'Gode': 17},
    'Mokadisho': {'Gode': 22},
    'Batu': {'Adama': 4, 'Buta Jirra': 2, 'Shashemene': 3},
    'Buta Jirra': {'Batu': 2, 'Worabe': 2},
    'Worabe': {'Buta Jirra': 2, 'Wolkite': 5, 'Hossana': 2},
    'Wolkite': {'Ambo': 6, 'Worabe': 5, 'Jimma': 8},
    'Jimma': {'Wolkite': 8, 'Bedelle': 9, 'Bonga': 4}, # Bedelle-Jimma assumed 9
    'Bedelle': {'Jimma': 9, 'Nekemte': 5, 'Gore': 6},  # Nekemte-Bedelle assumed 5
    'Nekemte': {'Ambo': 9, 'Bedelle': 5, 'Gimbi': 4},
    'Gimbi': {'Nekemte': 4, 'Dembi Dollo': 6},
    'Dembi Dollo': {'Gimbi': 6, 'Assosa': 12, 'Gambella': 4},
    'Assosa': {'Dembi Dollo': 12, 'Metekel': 8},       # Assosa-Metekel assumed 8
    'Gambella': {'Dembi Dollo': 4, 'Gore': 5},
    'Gore': {'Gambella': 5, 'Bedelle': 6, 'Tepi': 9},
    'Tepi': {'Gore': 9, 'Bonga': 8, 'Mezan Teferi': 4},
    'Bonga': {'Jimma': 4, 'Tepi': 8, 'Dawro': 10, 'Mezan Teferi': 4},
    'Mezan Teferi': {'Tepi': 4, 'Bonga': 4},
    'Dawro': {'Bonga': 10, 'Wolaita Sodo': 6},
    'Wolaita Sodo': {'Dawro': 6, 'Hossana': 4, 'Arba Minch': 5}, # Arba Minch-Wolaita assumed 5
    'Hossana': {'Worabe': 2, 'Wolaita Sodo': 4, 'Shashemene': 7},
    'Shashemene': {'Batu': 3, 'Hossana': 7, 'Hawassa': 1, 'Dodola': 3},
    'Hawassa': {'Shashemene': 1, 'Dilla': 3},
    'Dilla': {'Hawassa': 3, 'Bule Hora': 4},
    'Bule Hora': {'Dilla': 4, 'Yabello': 3},
    'Yabello': {'Bule Hora': 3, 'Konso': 3, 'Moyale': 6},
    'Moyale': {'Yabello': 6, 'Nairobi': 22},
    'Nairobi': {'Moyale': 22},
    'Konso': {'Yabello': 3, 'Arba Minch': 4},
    'Arba Minch': {'Wolaita Sodo': 5, 'Konso': 4, 'Basketo': 10},
    'Basketo': {'Arba Minch': 10, 'Bench Maji': 5},
    'Bench Maji': {'Basketo': 5, 'Juba': 22},
    'Juba': {'Bench Maji': 22},
    'Asella': {'Adama': 4, 'Assasa': 4},
    'Assasa': {'Asella': 4, 'Dodola': 1},
    'Dodola': {'Assasa': 1, 'Shashemene': 3, 'Bale': 13},
    'Bale': {'Dodola': 13, 'Goba': 18, 'Sof Oumer': 23, 'Liben': 11},
    'Goba': {'Bale': 18, 'Sof Oumer': 6},
    'Sof Oumer': {'Bale': 23, 'Goba': 6, 'Gode': 23},
    'Liben': {'Bale': 11}
}

# 2. CLASS IMPLEMENTATION
class UniformCostSearch:
    def __init__(self, graph):
        self.graph = graph

    def search(self, start, goal):
        """
        Standard Uniform Cost Search (UCS) to find the lowest cost path
        between a start node and a specific goal node.
        """
        # Priority Queue stores tuples: (cumulative_cost, current_node, path_list)
        pq = [(0, start, [start])]
        visited = set()

        while pq:
            cost, current, path = heapq.heappop(pq)

            if current == goal:
                return path, cost

            if current not in visited:
                visited.add(current)

                # Expand neighbors
                neighbors = self.graph.get(current, {})
                for neighbor, edge_cost in neighbors.items():
                    if neighbor not in visited:
                        new_cost = cost + edge_cost
                        new_path = path + [neighbor]
                        heapq.heappush(pq, (new_cost, neighbor, new_path))

        return None, float('inf')

    def multi_goal_search(self, start, goals):
        """
        Customized UCS to visit multiple goal states preserving local optimum.
        Strategy: From current location, use UCS to find the *nearest* unvisited
        goal state. Move there, mark as visited, and repeat.
        """
        current_node = start
        unvisited_goals = set(goals)
        full_path = [start]
        total_cost = 0

        print(f"Starting Multi-Goal Search from: {start}")
        print(f"Goals to visit: {goals}\n")

        while unvisited_goals:
            # We run a fresh UCS from the current node to find the CLOSEST unvisited goal
            # The 'goal' for this sub-search is "any node in unvisited_goals"
            nearest_goal, segment_path, segment_cost = self._find_nearest_goal(current_node, unvisited_goals)

            if nearest_goal is None:
                print(f"Cannot reach remaining goals: {unvisited_goals}")
                break

            # Update state
            # segment_path includes the start node, so we slice [1:] to avoid duplication in full_path
            full_path.extend(segment_path[1:])
            total_cost += segment_cost
            current_node = nearest_goal
            unvisited_goals.remove(nearest_goal)

            print(f"Visited: {nearest_goal} | Path segment: {segment_path} | Cost: {segment_cost}")

        return full_path, total_cost

    def _find_nearest_goal(self, start, goals):
        """
        Helper function: Runs UCS but stops as soon as ANY node in 'goals' is reached.
        """
        pq = [(0, start, [start])]
        visited = set()

        while pq:
            cost, current, path = heapq.heappop(pq)

            # If we hit one of our target goals, this is the nearest one (UCS property)
            if current in goals:
                return current, path, cost

            if current not in visited:
                visited.add(current)
                neighbors = self.graph.get(current, {})
                for neighbor, edge_cost in neighbors.items():
                    if neighbor not in visited:
                        heapq.heappush(pq, (cost + edge_cost, neighbor, path + [neighbor]))

        return None, [], 0

# 3. EXECUTION BLOCKS
solver = UniformCostSearch(ethiopia_map_weighted)

# --- 2.2 UCS Path from Addis Ababa to Lalibela ---
print("--- 2.2 Standard UCS: Addis Ababa -> Lalibela ---")
path, cost = solver.search("Addis Ababa", "Lalibela")
print(f"Optimal Path: {path}")
print(f"Total Cost: {cost}")
print("-" * 50)

# --- 2.3 Multi-Goal Search (Preserving Local Optimum) ---
print("\n--- 2.3 Customized Multi-Goal UCS ---")
goals_list = ["Axum", "Gondar", "Lalibela", "Babile", "Jimma", "Bale", "Sof Oumer", "Arba Minch"]
full_path, full_cost = solver.multi_goal_search("Addis Ababa", goals_list)

print(f"\nFinal Full Path: {full_path}")
print(f"Total Trip Cost: {full_cost}")

--- 2.2 Standard UCS: Addis Ababa -> Lalibela ---
Optimal Path: ['Addis Ababa', 'Debre Birhan', 'Debre Sina', 'Kemise', 'Dessie', 'Woldia', 'Lalibela']
Total Cost: 31
--------------------------------------------------

--- 2.3 Customized Multi-Goal UCS ---
Starting Multi-Goal Search from: Addis Ababa
Goals to visit: ['Axum', 'Gondar', 'Lalibela', 'Babile', 'Jimma', 'Bale', 'Sof Oumer', 'Arba Minch']

Visited: Jimma | Path segment: ['Addis Ababa', 'Ambo', 'Wolkite', 'Jimma'] | Cost: 19
Visited: Arba Minch | Path segment: ['Jimma', 'Wolkite', 'Worabe', 'Hossana', 'Wolaita Sodo', 'Arba Minch'] | Cost: 24
Visited: Bale | Path segment: ['Arba Minch', 'Wolaita Sodo', 'Hossana', 'Shashemene', 'Dodola', 'Bale'] | Cost: 32
Visited: Sof Oumer | Path segment: ['Bale', 'Sof Oumer'] | Cost: 23
Visited: Babile | Path segment: ['Sof Oumer', 'Gode', 'Kebri Dehar', 'Dega Habur', 'Jigjiga', 'Babile'] | Cost: 42
Visited: Lalibela | Path segment: ['Babile', 'Harar', 'Dire Dawa', 'Chiro', 'Awash', 'Gabi Ra